# Image emotion detection

Using the [FER-2013](https://www.kaggle.com/datasets/msambare/fer2013) dataset. First download and extract the dataset and place it in the working directory as `fer-2013-dataset`.

In [1]:
import struct;print(struct.calcsize("P") * 8)

import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img



train_datagen = ImageDataGenerator(rescale=1/255,
                                   zoom_range=0.3,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')



64


ModuleNotFoundError: No module named 'keras'